# Gerçek Zamanlı Vücut ve Yüz Tespiti

## Amaç
Canlı kamera görüntüsü üzerinden:
- Yüz tespiti (Haar Cascade)
- İnsan vücudu algılama (HOG + SVM)
- Poz tespiti (MediaPipe Pose – 33 vücut noktası)
işlemlerini gerçek zamanlı olarak gerçekleştirmek.


In [ ]:
!pip install mediapipe





In [ ]:
# --------------------
# Gerekli kütüphaneler
# --------------------
import cv2
import numpy as np
import urllib.request
import tempfile
import mediapipe as mp
from mediapipe.tasks.python import vision
from mediapipe.tasks.python.vision import pose_landmarker
from mediapipe.tasks.python.vision.core import BaseOptions, Image, ImageFormat

# --------------------
# Haar Cascade ile Yüz Tespiti
# --------------------
# Eğer XML dosyası yoksa indir
haarcascade_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
haarcascade_path = "haarcascade_frontalface_default.xml"

try:
    open(haarcascade_path, 'r')
except FileNotFoundError:
    urllib.request.urlretrieve(haarcascade_url, haarcascade_path)

# CascadeClassifier ile yükle
face_cascade = cv2.CascadeClassifier(haarcascade_path)

# --------------------
# HOG + SVM ile Vücut Algılama
# --------------------
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# --------------------
# MediaPipe PoseLandmarker (33 nokta + kemik çizimi)
# --------------------
# Modeli geçici klasöre indir
model_url = "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_full/float16/latest/pose_landmarker_full.task"
model_path = tempfile.gettempdir() + "/pose_landmarker_full.task"
urllib.request.urlretrieve(model_url, model_path)

# BaseOptions ile model yükle
base_options = BaseOptions(model_asset_path=model_path)

# PoseLandmarker seçenekleri
options = pose_landmarker.PoseLandmarkerOptions(
    base_options=base_options,
    running_mode=pose_landmarker.RunningMode.LIVE_STREAM,
    min_pose_detection_confidence=0.5,
    min_pose_tracking_confidence=0.5
)

# PoseLandmarker objesi
pose_landmarker_obj = pose_landmarker.PoseLandmarker.create_from_options(options)

# --------------------
# Kemik bağlantıları (33 nokta)
# --------------------
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7),
    (0, 4), (4, 5), (5, 6), (6, 8),
    (9, 10),
    (11, 12),
    (11, 13), (13, 15),
    (12, 14), (14, 16),
    (15, 17), (16, 18),
    (11, 23), (12, 24),
    (23, 24),
    (23, 25), (24, 26),
    (25, 27), (26, 28),
    (27, 29), (28, 30),
    (29, 31), (30, 32)
]

# --------------------
# Canlı Kamera Başlat
# --------------------
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    output = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # --------------------
    # Yüz Tespiti (Mavi Dikdörtgen)
    # --------------------
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces:
        cv2.rectangle(output, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(output, "Face", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)

    # --------------------
    # Vücut Algılama HOG (Yeşil Dikdörtgen)
    # --------------------
    bodies, _ = hog.detectMultiScale(frame, winStride=(4,4), padding=(8,8), scale=1.02)
    for (x, y, w, h) in bodies:
        cv2.rectangle(output, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(output, "Body", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    # --------------------
    # PoseLandmarker Tespiti ve Kemik Çizimi
    # --------------------
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_frame = Image(image_format=ImageFormat.SRGB, data=rgb_frame)

    # Algılama
    mp_result = pose_landmarker_obj.detect_for_video(mp_frame, timestamp_ms=int(cap.get(cv2.CAP_PROP_POS_MSEC)))

    if mp_result.pose_landmarks:
        for landmark in mp_result.pose_landmarks:
            h, w, _ = output.shape
            points = []
            # Noktaları çiz
            for pt in landmark:
                x_px = int(pt.x * w)
                y_px = int(pt.y * h)
                points.append((x_px, y_px))
                cv2.circle(output, (x_px, y_px), 5, (0,0,255), -1)

            # Kemik çizgileri
            for start_idx, end_idx in POSE_CONNECTIONS:
                if start_idx < len(points) and end_idx < len(points):
                    cv2.line(output, points[start_idx], points[end_idx], (0,255,255), 2)

    # --------------------
    # Sonuç Göster
    # --------------------
    cv2.imshow("Face + Body + PoseLandmarker", output)

    # 'q' ile çık
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



### Sonuçlar
- Yüzler mavi dikdörtgen ile tespit edildi
- İnsan vücutları yeşil dikdörtgen ile algılandı
- MediaPipe Pose ile 33 vücut noktası ve kemik bağlantıları çizildi
- Sistem gerçek zamanlı olarak çalışmaktadır


### Güçlü Yönler
- Gerçek zamanlı çalışma
- Üç farklı algılama tekniği birlikte kullanıldı
- Eğitim gerektirmeyen hazır modeller

### Zayıf Yönler
- HOG düşük ışıkta zorlanabilir
- Haar Cascade yüz açılarına duyarlıdır
